In [1]:
# import required libraries
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
import tensorflow as tf
tf.keras.utils.set_random_seed(1)

In [2]:
# loading files
from google.colab import files
uploaded = files.upload()

Saving BRCA.csv to BRCA.csv
Saving breast-cancer.csv to breast-cancer.csv


In [7]:
cancer_type_df = pd.read_csv('BRCA.csv').set_index('Patient_ID')
cancer_type_df.head()

Age  Gender  Protein1  Protein2  Protein3  Protein4  \
Patient_ID                                                           
TCGA-D8-A1XD  36.0  FEMALE  0.080353   0.42638   0.54715  0.273680   
TCGA-EW-A1OX  43.0  FEMALE -0.420320   0.57807   0.61447 -0.031505   
TCGA-A8-A079  69.0  FEMALE  0.213980   1.31140  -0.32747 -0.234260   
TCGA-D8-A1XR  56.0  FEMALE  0.345090  -0.21147  -0.19304  0.124270   
TCGA-BH-A0BF  56.0  FEMALE  0.221550   1.90680   0.52045 -0.311990   

             Tumour_Stage                      Histology ER status PR status  \
Patient_ID                                                                     
TCGA-D8-A1XD          III  Infiltrating Ductal Carcinoma  Positive  Positive   
TCGA-EW-A1OX           II             Mucinous Carcinoma  Positive  Positive   
TCGA-A8-A079          III  Infiltrating Ductal Carcinoma  Positive  Positive   
TCGA-D8-A1XR           II  Infiltrating Ductal Carcinoma  Positive  Positive   
TCGA-BH-A0BF           II  Infiltrating Ductal Carcinoma  Positive  Positive   

             HER2 status                 Surgery_type Date_of_Surgery  \
Patient_ID                                                              
TCGA-D8-A1XD    Negative  Modified Radical Mastectomy       15-Jan-17   
TCGA-EW-A1OX    Negative                   Lumpectomy       26-Apr-17   
TCGA-A8-A079    Negative                        Other       08-Sep-17   
TCGA-D8-A1XR    Negative  Modified Radical Mastectomy       25-Jan-17   
TCGA-BH-A0BF    Negative                        Other       06-May-17   

             Date_of_Last_Visit Patient_Status  
Patient_ID                                      
TCGA-D8-A1XD          19-Jun-17          Alive  
TCGA-EW-A1OX          09-Nov-18           Dead  
TCGA-A8-A079          09-Jun-18          Alive  
TCGA-D8-A1XR          12-Jul-17          Alive  
TCGA-BH-A0BF          27-Jun-19           Dead

In [8]:
cancer_df = pd.read_csv('breast-cancer.csv').set_index('id')
cancer_df.head()

diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
id                                                                         
842302           M        17.99         10.38          122.80     1001.0   
842517           M        20.57         17.77          132.90     1326.0   
84300903         M        19.69         21.25          130.00     1203.0   
84348301         M        11.42         20.38           77.58      386.1   
84358402         M        20.29         14.34          135.10     1297.0   

          smoothness_mean  compactness_mean  concavity_mean  \
id                                                            
842302            0.11840           0.27760          0.3001   
842517            0.08474           0.07864          0.0869   
84300903          0.10960           0.15990          0.1974   
84348301          0.14250           0.28390          0.2414   
84358402          0.10030           0.13280          0.1980   

          concave points_mean  symmetry_mean  ...  radius_worst  \
id                                            ...                 
842302                0.14710         0.2419  ...         25.38   
842517                0.07017         0.1812  ...         24.99   
84300903              0.12790         0.2069  ...         23.57   
84348301              0.10520         0.2597  ...         14.91   
84358402              0.10430         0.1809  ...         22.54   

          texture_worst  perimeter_worst  area_worst  smoothness_worst  \
id                                                                       
842302            17.33           184.60      2019.0            0.1622   
842517            23.41           158.80      1956.0            0.1238   
84300903          25.53           152.50      1709.0            0.1444   
84348301          26.50            98.87       567.7            0.2098   
84358402          16.67           152.20      1575.0            0.1374   

          compactness_worst  concavity_worst  concave points_worst  \
id                                                                   
842302               0.6656           0.7119                0.2654   
842517               0.1866           0.2416                0.1860   
84300903             0.4245           0.4504                0.2430   
84348301             0.8663           0.6869                0.2575   
84358402             0.2050           0.4000                0.1625   

          symmetry_worst  fractal_dimension_worst  
id                                                 
842302            0.4601                  0.11890  
842517            0.2750                  0.08902  
84300903          0.3613                  0.08758  
84348301          0.6638                  0.17300  
84358402          0.2364                  0.07678  

[5 rows x 31 columns]

In [6]:
cancer_type_df['Histology'].value_counts()

Infiltrating Ductal Carcinoma     233
Infiltrating Lobular Carcinoma     89
Mucinous Carcinoma                 12
Name: Histology, dtype: int64